In [5]:
%pip install --upgrade pandas
%pip install --upgrade nltk

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
from collections import Counter
import itertools 
import re
# import contractions

import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('treebank')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/leomoore/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/leomoore/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leomoore/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /Users/leomoore/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /Users/leomoore/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leomoore/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
df_a = pd.read_csv("fake_job_postings_A.csv")
df_b = pd.read_csv("fake_job_postings_B.csv")

df = pd.concat([df_a, df_b], ignore_index=True)

df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,...,Unnamed: 8,0,0.1,0.2,Full-time,Unnamed: 13,Unnamed: 14,Management Consulting,Unnamed: 16,0.3
0,1.0,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
global_stopwords = nltk.corpus.stopwords.words("english")
THRESHOLD = 2

df["description"] = df["description"].str.replace(r"[^\w\s]", "", regex=True)
df["description"] = df["description"].str.lower()
words_all = [nltk.word_tokenize(s) for s in df["description"] if isinstance(s, str)]
words_all = list(itertools.chain.from_iterable(words_all))
words_all = [w for w in words_all if len(w)> THRESHOLD and w not in global_stopwords]
words_all


['food52',
 'fastgrowing',
 'james',
 'beard',
 'awardwinning',
 'online',
 'food',
 'community',
 'crowdsourced',
 'curated',
 'recipe',
 'hub',
 'currently',
 'interviewing',
 'full',
 'parttime',
 'unpaid',
 'interns',
 'work',
 'small',
 'team',
 'editors',
 'executives',
 'developers',
 'new',
 'york',
 'city',
 'headquartersreproducing',
 'andor',
 'repackaging',
 'existing',
 'food52',
 'content',
 'number',
 'partner',
 'sites',
 'huffington',
 'post',
 'yahoo',
 'buzzfeed',
 'various',
 'content',
 'management',
 'systemsresearching',
 'blogs',
 'websites',
 'provisions',
 'food52',
 'affiliate',
 'programassisting',
 'daytoday',
 'affiliate',
 'program',
 'support',
 'screening',
 'affiliates',
 'assisting',
 'affiliate',
 'inquiriessupporting',
 'amp',
 'events',
 'neededhelping',
 'office',
 'administrative',
 'work',
 'filing',
 'mailing',
 'preparing',
 'meetingsworking',
 'developers',
 'document',
 'bugs',
 'suggest',
 'improvements',
 'sitesupporting',
 'marketing',
 '

In [42]:
counts = Counter(words_all)
counts.most_common(20)
dffreq = pd.DataFrame(counts.most_common(), columns=["Term", "Frequency"])
dffreq.head(10)

,Term,Frequency
0,team,8631
1,work,6784
2,business,5386
3,new,4840
4,experience,4695
5,company,4591
6,customer,4572
7,sales,4478
8,looking,4300
9,development,4010
